## Image Enhancement

In [1]:
import cv2
import numpy as np

def preprocess_image(image_path, save_path):
    # Read the image
    original_image = cv2.imread('original/lens_wipe.jpg')

    # Resize the image for consistency
    resized_image = cv2.resize(original_image, (500, 500))

    # Apply Gaussian blur for smoothing
    blurred_image = cv2.GaussianBlur(resized_image, (5, 5), 0)

    # Display the original and preprocessed images
    cv2.imshow('Original Image', resized_image)
    cv2.imshow('Gaussian Blur', blurred_image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the preprocessed images
    save_folder = 'enhanced'
    if not cv2.os.path.exists(save_folder):
        cv2.os.makedirs(save_folder)

    # Save only the Gaussian blur and histogram equalization images
    cv2.imwrite(f'{save_folder}/gaussian_{save_path}', blurred_image)

# Paths to your images and names for saving
object_image_paths = [('enhanced/lens_wipe.jpg', 'lens_wipe_enhanced.jpg')]

# Process object images
for path, save_name in object_image_paths:
    preprocess_image(path, save_name)


## Edge Detection Filter

In [2]:
import cv2
import os

def prewitt_edge_detection(image):
    kernel_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    kernel_y = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
    gradient_x = cv2.filter2D(image, cv2.CV_64F, kernel_x)
    gradient_y = cv2.filter2D(image, cv2.CV_64F, kernel_y)
    edges = np.sqrt(gradient_x**2 + gradient_y**2)
    edges = cv2.normalize(edges, None, 0, 255, cv2.NORM_MINMAX)
    edges = np.uint8(edges)
    return edges

def laplacian_edge_detection(image):
    edges = cv2.Laplacian(image, cv2.CV_64F)
    edges = cv2.normalize(edges, None, 0, 255, cv2.NORM_MINMAX)
    edges = np.uint8(edges)
    return edges

def sobel_edge_detection(image):
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    edges = np.sqrt(gradient_x**2 + gradient_y**2)
    edges = cv2.normalize(edges, None, 0, 255, cv2.NORM_MINMAX)
    edges = np.uint8(edges)
    return edges

def canny_edge_detection(image):
    edges = cv2.Canny(image, 50, 150)
    return edges

def apply_edge_detection(image_path, filter_name, save_folder):
    # Read the image
    original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply the specified edge detection filter
    if filter_name == 'prewitt':
        edges = prewitt_edge_detection(original_image)
    elif filter_name == 'laplacian':
        edges = laplacian_edge_detection(original_image)
    elif filter_name == 'sobel':
        edges = sobel_edge_detection(original_image)
    elif filter_name == 'canny':
        edges = canny_edge_detection(original_image)
    else:
        raise ValueError('Invalid filter name')

    # Save the processed image
    save_path = os.path.join(save_folder, f'{filter_name}_{os.path.basename(image_path)}')
    cv2.imwrite(save_path, edges)

# Paths to images and create folder for saving
image_paths = ['enhanced/gaussian_lens_wipe_enhanced.jpg']
save_folder = 'filter'

# Apply edge detection for each filter
for image_path in image_paths:
    for filter_name in ['prewitt', 'laplacian', 'sobel', 'canny']:
        apply_edge_detection(image_path, filter_name, save_folder)


## Edge detection on main object

In [3]:
import cv2
import numpy as np

def prewitt_edge_detection(image):
    # Convert the image to grayscale if not already
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Prewitt edge detection
    kernel_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    kernel_y = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
    gradient_x = cv2.filter2D(image, cv2.CV_64F, kernel_x)
    gradient_y = cv2.filter2D(image, cv2.CV_64F, kernel_y)
    edges = np.sqrt(gradient_x**2 + gradient_y**2)

    # Normalize edges to range [0, 255]
    edges = cv2.normalize(edges, None, 0, 255, cv2.NORM_MINMAX)
    edges = np.uint8(edges)

    return edges

def detect_and_draw_bounding_box(main_object_path, image_paths, save_folder):
    # Read the main object image
    main_object = cv2.imread(main_object_path)
    main_object_gray = cv2.cvtColor(main_object, cv2.COLOR_BGR2GRAY)

    # Prewitt edge detection on the main object
    edges_main_object = prewitt_edge_detection(main_object)

    # Create ORB detector
    orb = cv2.ORB_create(nfeatures=350)

    # Find the keypoints and descriptors with ORB
    keypoints_main_object, descriptors_main_object = orb.detectAndCompute(main_object_gray, None)

    # Create BFMatcher (Brute Force Matcher)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Loop through images and detect the main object
    for image_path in image_paths:
        # Read the image
        image = cv2.imread(image_path)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Prewitt edge detection on the image
        edges_image = prewitt_edge_detection(image)

        # Find the keypoints and descriptors with ORB
        keypoints_image, descriptors_image = orb.detectAndCompute(image_gray, None)

        # Match descriptors using BFMatcher
        matches = bf.match(descriptors_main_object, descriptors_image)

        # Sort them in ascending order of distance
        matches = sorted(matches, key=lambda x: x.distance)

        # Set a threshold for considering a match
        threshold = 30  # Adjust this threshold as needed
        good_matches = [match for match in matches if match.distance < threshold]

        if len(good_matches) > 10:
            # Extract matched keypoints
            src_pts = np.float32([keypoints_main_object[match.queryIdx].pt for match in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([keypoints_image[match.trainIdx].pt for match in good_matches]).reshape(-1, 1, 2)

            # Find the homography matrix
            M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, ransacReprojThreshold=3.0)

            # Get the coordinates of the main object in the image
            h, w = edges_main_object.shape
            obj_corners = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
            dst_corners = cv2.perspectiveTransform(obj_corners, M)

            # Draw a bounding box around the main object
            image_with_box = image.copy()
            cv2.polylines(image_with_box, [np.int32(dst_corners)], True, (0, 255, 0), 2)

            # Save the result with bounding box
            save_path = cv2.os.path.join(save_folder, f'detected_{cv2.os.path.basename(image_path)}')
            cv2.imwrite(save_path, image_with_box)
            print(f'Main object detected in {image_path}')
        else:
            # Save the original image without bounding box
            save_path = cv2.os.path.join(save_folder, cv2.os.path.basename(image_path))
            cv2.imwrite(save_path, image)
            print(f'Main object not detected in {image_path}')

# Paths to images and save folder
main_object_path = 'enhanced/gaussian_lens_wipe_enhanced.jpg'
image_paths = ['original/lens_wipe_random1.jpg', 'original/lens_wipe_random2.jpg', 'original/random1.jpg', 'original/random2.jpg']
save_folder = 'output'

# Detect and draw bounding box using ORB feature matching
detect_and_draw_bounding_box(main_object_path, image_paths, save_folder)


Main object detected in original/lens_wipe_random1.jpg
Main object detected in original/lens_wipe_random2.jpg
Main object not detected in original/random1.jpg
Main object not detected in original/random2.jpg
